# EC-EARTH CMIP6 2004-2014

Notebook environment to migrate NetCDF files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Import standard packages
import os
import pathlib
import sys

import numpy as np
#import geopandas as gpd
 
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import glob
import itertools
import json
import copy
from itertools import chain
from shapely import wkb

# Import custom functionality
# sys.path.append('c:/Windows/System32/coclicodata/src')#CLENMAR ADDED

from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy
#P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\01_Hazards\01_Present\Extreme_Wave_Height
# Define (local and) remote drives
# gca_data_dir = pathlib.Path.home().joinpath(r"OneDrive - Stichting Deltares\Documents\GitHub")
gca_data_dir = p_drive.joinpath("11209197-018-global-coastal-atlas", r"MSc_students\ClenmarRowe\Data\All_Datasets", r"Orig_Datasets\01_Hazards\02_Future")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"AppData\Local\miniconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml" ###Changed from anaconda to miniconda  - added the new package udunits file
    )
)


<IPython.core.display.Javascript object>

In [3]:
# Project paths & files (manual input)
dataset_dir = gca_data_dir.joinpath(r"Extreme_Wave_Height\GCM_Hindcast_2004-2014")
dataset_dir_path = gca_data_dir.joinpath(r"Extreme_Wave_Height\GCM_Hindcast_2004-2014","EC-EARTH_WW3_3Hourly_2004-2014.nc")
CF_dir = gca_data_dir.joinpath(r"Extreme_Wave_Height\GCM_Hindcast_2004-2014","CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

In [4]:
#For SSP1-26

import glob

# Specify the path to the directory containing the netCDF files
dir="hindcast"

# Get a list of all netCDF files in the specified directory
files = sorted(glob.glob(os.path.join(dataset_dir , dir,  "*.nc")))

# Open datasets using lazily in Chunks
datasets = [xr.open_dataset(f, chunks={'time':  1}) for f in files]


<IPython.core.display.Javascript object>

In [5]:

# Concatenate the datasets along the time dimension
merged_hindcast = xr.concat(datasets, dim='time')#.compute()

# Save the merged dataset to a new netCDF file in the current directory
# merged.to_netcdf('merged_output.nc')
merged_hindcast

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 361, time: 32144)
Coordinates:
  * longitude  (longitude) float32 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
  * latitude   (latitude) float32 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * time       (time) datetime64[ns] 2004-01-01 ... 2014-12-31T21:00:00
Data variables:
    MAPSTA     (time, latitude, longitude) int16 dask.array<chunksize=(248, 361, 720), meta=np.ndarray>
    hs         (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
Attributes: (12/15)
    WAVEWATCH_III_version_number:  6.07
    WAVEWATCH_III_switches:        F90 NOGRB NC4 SCRIP SHRD PR3 UQ FLX4 LN1 S...
    product_name:                  ww3.200401_hs.nc
    area:                          Global regular grid output
    latitude_resolution:           0.
    longitude_resolution:          0.
    ...                            ...
    easternmost_longitude:         360.
    minimum_altitude:              -12000 m
    maximum_altitude:              9000 m
    altitude_resolution:           n/a
    start_date:                    2004-01-01 00:00:00
    stop_date:                     2004-01-31 21:00:00

<IPython.core.display.Javascript object>

In [6]:
ds=merged_hindcast.drop_vars("MAPSTA")

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [7]:
%%capture cap --no-stderr
# check original CF compliancy (for first file)

check_compliancy(testfile=files[0], 
                 working_dir=CF_dir
                 )

<IPython.core.display.Javascript object>

In [8]:
# # save original CF compliancy (for first file)
save_compliancy(cap, testfile=dataset_dir_path, working_dir=CF_dir)

{'P:\\11209197-018-global-coastal-atlas\\MSc_students\\ClenmarRowe\\Data\\All_Datasets\\Orig_Datasets\\01_Hazards\\02_Future\\Extreme_Wave_Height\\GCM_Hindcast_2004-2014\\hindcast\\ww3.200401_hs.nc': {'warnings': '1', 'errors': '2'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [9]:
import json

# NetCDF attribute alterations by means of metadata template
metadata_json=dataset_dir.joinpath("metadata_EC-Earth-2004-2014.json")

f_global = open(metadata_json)
meta_global = json.load(f_global)


for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    ds.attrs[attr_name] = attr_val

ds.attrs['Conventions'] = "CF-1.8"
ds

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 361, time: 32144)
Coordinates:
  * longitude  (longitude) float32 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
  * latitude   (latitude) float32 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * time       (time) datetime64[ns] 2004-01-01 ... 2014-12-31T21:00:00
Data variables:
    hs         (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
Attributes: (12/36)
    WAVEWATCH_III_version_number:  6.07
    WAVEWATCH_III_switches:        F90 NOGRB NC4 SCRIP SHRD PR3 UQ FLX4 LN1 S...
    product_name:                  ww3.200401_hs.nc
    area:                          Global regular grid output
    latitude_resolution:           0.
    longitude_resolution:          0.
    ...                            ...
    DOI:                           
    LONG_NAME:                     SIGNIFICANT_WAVE_HEIGHT_2004_2014_CMIP6_EC...
    UNITS:                         meters
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8

<IPython.core.display.Javascript object>

# Slice to reduce size of CF checked NetCDF

In [10]:
ds_slice=ds.sel(time=slice('2004-01-01','2004-01-02'))
ds_slice

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 361, time: 16)
Coordinates:
  * longitude  (longitude) float32 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
  * latitude   (latitude) float32 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * time       (time) datetime64[ns] 2004-01-01 ... 2004-01-02T21:00:00
Data variables:
    hs         (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
Attributes: (12/36)
    WAVEWATCH_III_version_number:  6.07
    WAVEWATCH_III_switches:        F90 NOGRB NC4 SCRIP SHRD PR3 UQ FLX4 LN1 S...
    product_name:                  ww3.200401_hs.nc
    area:                          Global regular grid output
    latitude_resolution:           0.
    longitude_resolution:          0.
    ...                            ...
    DOI:                           
    LONG_NAME:                     SIGNIFICANT_WAVE_HEIGHT_2004_2014_CMIP6_EC...
    UNITS:                         meters
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8

<IPython.core.display.Javascript object>

In [11]:
# var=ds['swh'].isel(time=0)
# var.plot()
# plt.show()

<IPython.core.display.Javascript object>

In [12]:
# Write the xarray dataset to a netCDF file
#Compliant netcdf
dataset_dir_path_CF=pathlib.Path().joinpath(str(dataset_dir_path)[0:-3]+"_CF"+".nc")


# ds.to_netcdf(path=dataset_dir_path_CF)
#ds_slice.to_netcdf(path=r"C:\Users\rowe\OneDrive - Stichting Deltares\Desktop\delete\era5_coastal_Hs_all_years_CF.nc")
ds_slice.to_netcdf(path=dataset_dir_path_CF)

<IPython.core.display.Javascript object>

### Check CF compliancy modified NetCDF files

In [13]:
# open datasets (only first file, rest is the same)
ds_slice = xr.open_dataset(dataset_dir_path_CF)

# check original dataset
ds_slice

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 361, time: 16)
Coordinates:
  * longitude  (longitude) float32 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
  * latitude   (latitude) float32 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * time       (time) datetime64[ns] 2004-01-01 ... 2004-01-02T21:00:00
Data variables:
    hs         (time, latitude, longitude) float32 ...
Attributes: (12/36)
    WAVEWATCH_III_version_number:  6.07
    WAVEWATCH_III_switches:        F90 NOGRB NC4 SCRIP SHRD PR3 UQ FLX4 LN1 S...
    product_name:                  ww3.200401_hs.nc
    area:                          Global regular grid output
    latitude_resolution:           0.
    longitude_resolution:          0.
    ...                            ...
    DOI:                           
    LONG_NAME:                     SIGNIFICANT_WAVE_HEIGHT_2004_2014_CMIP6_EC...
    UNITS:                         meters
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8

<IPython.core.display.Javascript object>

In [14]:
%%capture cap --no-stderr
# check original CF compliancy (for first file)

check_compliancy(testfile=dataset_dir_path_CF, 
                 working_dir=CF_dir
                 )

<IPython.core.display.Javascript object>

In [15]:
# save original CF compliancy (for first file)
save_compliancy(cap, testfile=dataset_dir_path_CF, working_dir=CF_dir)

{'P:\\11209197-018-global-coastal-atlas\\MSc_students\\ClenmarRowe\\Data\\All_Datasets\\Orig_Datasets\\01_Hazards\\02_Future\\Extreme_Wave_Height\\GCM_Hindcast_2004-2014\\EC-EARTH_WW3_3Hourly_2004-2014_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

### Write data to Zarr files

In [16]:
from dask.diagnostics import ProgressBar


# # export to zarr in write mode (to overwrite if exists)

with ProgressBar():
    ds.to_zarr(str(dataset_dir_path).replace(".nc", ".zarr"), mode="w")

[########################################] | 100% Completed | 19m 51s


<IPython.core.display.Javascript object>

In [17]:
# # export to zarr in write mode (to overwrite if exists)
# ds.to_zarr(str(dataset_dir_path).replace(".nc", ".zarr"), mode="w")

<IPython.core.display.Javascript object>